# Sophia Hart
# Recipe Recommender with Ingredient Optimization Database
## Capstone Project: Google x Kaggle GenAI 5-day Intensive Course, 2025
### April 20th, 2025
### sophia.t.hart@gmail.com

# Problem 
Meals are important—for the creativity in planning and cooking, the joy of eating good food (especially if you are sharing it with your loved ones), and the nutrients they provide for healthy living. For quality living, we like fresh ingredients and exploring recipes. This does not come easily with time constraints. In this project, I built a database of ingredient inventory specific to the household. After taking the Intensive 5-Day GenAI course that Google and Kaggle put together, I attempted to apply GenAI to solve this optimization problem. I built a GenAI agent to interact with the ingredient database. If there is a recipe database, this application would call the other agents, apply optimization techniques, and recommend a recipe to a chef.  



# Use Case
This application's purpose is for families and individuals to eat healthily at home daily; however, it can be generalized to be used in restaurants with different customers if the restaurants have chefs with diverse cooking talents. 

The user's ingredients inventory is stored in a database. The GenAI agent converses with the user to evaluate the user's meal preference on that day/moment, looks at the user's inventory, and plans a meal for the user. The inventory input can come from three different sources: 1. The user inputs one item at a time using a UI that interfaces the database. 2. The user provides the store name. When the items are checked out, if the store has a partnership with the owner of this app, the checked-out items will be stored in the user's database. 3. This applied to both in-store and online checked-out, although online may have a different interface rather than providing the store's name.

# 1. Setup

Start by installing and importing Python SDK

In [ ]:
!pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
!pip install -U -q "google-genai==1.7.0"

In [ ]:
from google import genai
from google.genai import types

genai.__version__

To run the following cell, your API key must be stored in a Kaggle secret named GOOGLE_API_KEY (go to Add-ons).

In [ ]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

Define an automated retry policy

In [ ]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

# 2. Building a User's Ingredient Inventory Database
## Interacting with an SQLite database using SQLAlchemy Core

Use SQLAlchemy Core to create tables and insert multiple lines of data.
Store database in Kaggle's working directory.

In [ ]:
import os
import sqlalchemy
from sqlalchemy import create_engine, text
from datetime import datetime

# --- Define the path within Kaggle's writable directory ---
db_directory = "/kaggle/working/"  # Standard writable directory in Kaggle notebooks
db_filename = "ingredient_inventory.db" 
db_path = os.path.join(db_directory, db_filename)

# --- Create the SQLAlchemy connection string ---
# Use triple slashes for an absolute path on Linux-like systems (which Kaggle uses)
connection_string = f'sqlite:///{db_path}'

print(f"Using database path: {db_path}") # Good for verification

### Table locations
Use engine to create tables (locations table and ingredients table) and insert data into the tables. 

In [ ]:
# Parameterized CREATE statement
sql_create_table = text("""
    CREATE TABLE IF NOT EXISTS locations (
      location_id INTEGER PRIMARY KEY,
      location_name TEXT
    )
""")

# Parameterized INSERT statement
sql_insert = text("""
    INSERT OR IGNORE INTO locations (location_id, location_name)
    VALUES (:id, :name)
""")

# Data as a LIST OF DICTIONARIES
data_to_insert = [
    {'id': 1, 'name': 'Refrigerator - Main Shelf'},
    {'id': 2, 'name': 'Pantry Shelf A (Dry Goods)'},
    {'id': 3, 'name': 'Freezer - Top Drawer'},
    {'id': 4, 'name': 'Countertop Fruit Bowl'},
    {'id': 5, 'name': 'Spice Rack'},
    {'id': 6, 'name': 'Pantry Shelf B (Root Veg/Other)'},
    {'id': 7, 'name': 'Freezer - Bottom Drawer'},
    {'id': 8, 'name': 'Fridge - Crisper Drawer'},
    {'id': 9, 'name': 'Fridge - Door Shelf (Condiments)'}
]

# Execute using a connection (manages transactions)
try:
    # No need for check_same_thread=False unless you explicitly use threading
    # within your *notebook code itself* to access the *same connection object*.
    # Kaggle's environment handles the notebook execution thread.
    engine = create_engine(connection_string)
    
    with engine.connect() as connection:
        # Use connection to a create table
        connection.execute(sql_create_table)
        # Use executemany for inserting multiple rows efficiently
        connection.execute(sql_insert, data_to_insert)
        connection.commit() # Commit the transaction
    print("Data inserted successfully!")
except Exception as e:
    print(f"An error occurred: {e}")

### Table ingredients
Define data cleaning function to ensure dates are in the correct format.


In [ ]:
# --- Helper function to standardize dates ---
def standardize_date(date_str):
    """Attempts to parse common date formats and return 'YYYY-MM-DD'."""
    if not date_str:
        return None
    formats_to_try = [
        "%Y-%m-%d",  # Already correct
        "%m/%d/%Y",  # e.g., 04/17/2025
        "%m/%d/%y",  # e.g., 04/17/25 (less likely here, but good to have)
        "%Y/%m/%d",
        "%d-%b-%Y", # e.g., 17-Apr-2025
        # Add more formats if needed
    ]
    # Handle single-digit month/day by attempting direct parse first
    try:
        # Try parsing M/D/YYYY or M/D/YY variations directly if slashes exist
        if '/' in date_str:
            parts = date_str.split('/')
            if len(parts) == 3:
                 # Pad month and day if needed
                 month = parts[0].zfill(2)
                 day = parts[1].zfill(2)
                 year = parts[2]
                 # Handle 2-digit year (assume 20xx)
                 if len(year) == 2:
                     year = "20" + year
                 elif len(year) != 4:
                     raise ValueError("Invalid year format") # Or handle differently

                 parsed_date = datetime.strptime(f"{month}/{day}/{year}", "%m/%d/%Y")
                 return parsed_date.strftime('%Y-%m-%d')
    except ValueError:
        pass # If direct parsing fails, try other formats

    # Try standard formats
    for fmt in formats_to_try:
        try:
            parsed_date = datetime.strptime(date_str, fmt)
            return parsed_date.strftime('%Y-%m-%d')
        except ValueError:
            continue # Try the next format

    # If no format matches, return original or raise error/log warning
    print(f"Warning: Could not parse date '{date_str}'. Inserting as is or as NULL.")
    # Decide how to handle unparseable dates: return None, return original, raise error
    return None # Or return date_str if column is TEXT

Define create and insert SQL queries. Put data in a list of dictionaries.

In [ ]:
# Define CREATE statement (Optionally add FOREIGN KEY)
sql_create_table = text("""
  CREATE TABLE IF NOT EXISTS ingredients (
      ingredient_id INTEGER PRIMARY KEY,
      ingredient_name TEXT NOT NULL, -- Added NOT NULL constraint
      quantity REAL, -- Changed to REAL to handle potential non-integer quantities
      unit TEXT,
      category TEXT,
      expiration_date TEXT, -- Changed to TEXT for flexibility OR keep DATE if data is cleaned
      location_id INTEGER,
      -- Uncomment and ensure locations table exists if you need this constraint:
      FOREIGN KEY (location_id) REFERENCES locations (location_id)
  )
""")

# --- Define the ALTER TABLE statement ---
sql_add_column = text("""
    ALTER TABLE ingredients
    ADD COLUMN category TEXT
""")

sql_insert = text("""
    INSERT OR REPLACE INTO ingredients (ingredient_id, ingredient_name, quantity, unit, category, expiration_date, location_id)
    VALUES (:id, :name, :qty, :unit, :cat, :exp_date, :loc_id)
""")

# Data as a LIST OF DICTIONARIES
data_to_insert_raw = [
    {'id': 1, 'name': 'avocado', 'qty': 6, 'unit': 'pcs', 'cat': 'Produce', 'exp_date': '04/17/2025', 'loc_id': 4}, # Corrected spelling
    {'id': 2, 'name': 'grape tomato', 'qty': 1, 'unit': 'box', 'cat': 'Produce', 'exp_date': '04/18/2025', 'loc_id': 1},
    {'id': 3, 'name': 'eggplant', 'qty': 2, 'unit': 'pcs', 'cat': 'Produce', 'exp_date': '4/12/2025', 'loc_id': 8},
    {'id': 4, 'name': 'cauliflower', 'qty': 1, 'unit': 'head', 'cat': 'Produce', 'exp_date': '4/9/2025', 'loc_id': 8}, # Corrected spelling
    {'id': 17, 'name': 'Lettuce (Romaine)', 'qty': 1, 'unit': 'head', 'cat': 'Produce', 'exp_date': '2025-04-12', 'loc_id': 8},
    {'id': 18, 'name': 'Bell Pepper (Red)', 'qty': 2, 'unit': 'pcs', 'cat': 'Produce', 'exp_date': '2025-04-15', 'loc_id': 8},
    {'id': 19, 'name': 'Carrots', 'qty': 5, 'unit': 'pcs', 'cat': 'Produce', 'exp_date': '2025-05-08', 'loc_id': 8},
    {'id': 20, 'name': 'Apples (Gala)', 'qty': 4, 'unit': 'pcs', 'cat': 'Produce', 'exp_date': '2025-04-16', 'loc_id': 4},
    {'id': 21, 'name': 'Bananas', 'qty': 3, 'unit': 'pcs', 'cat': 'Produce', 'exp_date': '2025-04-12', 'loc_id': 4},
    {'id': 22, 'name': 'Onions (Yellow)', 'qty': 2, 'unit': 'pcs', 'cat': 'Produce', 'exp_date': '2025-05-10', 'loc_id': 6},
    {'id': 23, 'name': 'Garlic', 'qty': 1, 'unit': 'head', 'cat': 'Produce', 'exp_date': '2025-05-15', 'loc_id': 6},
    {'id': 24, 'name': 'Potatoes (Russet)', 'qty': 1500, 'unit': 'g', 'cat': 'Produce', 'exp_date': '2025-05-10', 'loc_id': 6},
    # Meat & Poultry & Seafood
    {'id': 5, 'name': 'NY strip', 'qty': 2, 'unit': 'pcs', 'cat': 'Meat', 'exp_date': '4/21/2025', 'loc_id': 7},
    {'id': 25, 'name': 'Chicken Breast', 'qty': 500, 'unit': 'g', 'cat': 'Poultry', 'exp_date': '2025-05-25', 'loc_id': 7},
    {'id': 26, 'name': 'Ground Beef (80/20)', 'qty': 450, 'unit': 'g', 'cat': 'Meat', 'exp_date': '2025-06-15', 'loc_id': 7},
    {'id': 27, 'name': 'Salmon Fillet', 'qty': 300, 'unit': 'g', 'cat': 'Seafood', 'exp_date': '2025-06-20', 'loc_id': 7},
    # Frozen Goods (including meals)
    {'id': 6, 'name': 'orange chicken', 'qty': 1, 'unit': 'box', 'cat': 'Frozen Goods', 'exp_date': '5/1/2025', 'loc_id': 3},
    {'id': 7, 'name': 'crab cake', 'qty': 1, 'unit': 'box', 'cat': 'Frozen Goods', 'exp_date': '5/3/2025', 'loc_id': 3},
    {'id': 38, 'name': 'Frozen Peas', 'qty': 400, 'unit': 'g', 'cat': 'Frozen Goods', 'exp_date': '2025-10-30', 'loc_id': 3},
    {'id': 39, 'name': 'Frozen Berries (Mixed)', 'qty': 500, 'unit': 'g', 'cat': 'Frozen Goods', 'exp_date': '2025-07-15', 'loc_id': 3},
    {'id': 40, 'name': 'Frozen Pizza', 'qty': 1, 'unit': 'pcs', 'cat': 'Frozen Goods', 'exp_date': '2025-05-28', 'loc_id': 3},
    # Pantry Staples (Dry Goods)
    {'id': 8, 'name': 'coffee bean', 'qty': 1, 'unit': 'bag', 'cat': 'Beverages', 'exp_date': '4/30/2025', 'loc_id': 2}, # Could be Pantry, using Beverages
    {'id': 9, 'name': 'sugar', 'qty': 1, 'unit': 'jar', 'cat': 'Pantry Staples', 'exp_date': '4/30/2025', 'loc_id': 2},
    {'id': 29, 'name': 'All-Purpose Flour', 'qty': 1200, 'unit': 'g', 'cat': 'Pantry Staples', 'exp_date': '2026-05-01', 'loc_id': 2},
    {'id': 30, 'name': 'Granulated Sugar', 'qty': 800, 'unit': 'g', 'cat': 'Pantry Staples', 'exp_date': '2026-10-01', 'loc_id': 2},
    {'id': 31, 'name': 'Pasta (Spaghetti)', 'qty': 450, 'unit': 'g', 'cat': 'Pantry Staples', 'exp_date': '2026-08-30', 'loc_id': 2},
    {'id': 32, 'name': 'Rice (Basmati)', 'qty': 900, 'unit': 'g', 'cat': 'Pantry Staples', 'exp_date': '2026-11-15', 'loc_id': 2},
    {'id': 36, 'name': 'Rolled Oats', 'qty': 600, 'unit': 'g', 'cat': 'Pantry Staples', 'exp_date': '2026-04-22', 'loc_id': 6},
    # Canned Goods
    {'id': 33, 'name': 'Canned Diced Tomatoes', 'qty': 2, 'unit': 'cans', 'cat': 'Canned Goods', 'exp_date': '2026-03-10', 'loc_id': 2},
    {'id': 34, 'name': 'Canned Chickpeas', 'qty': 1, 'unit': 'cans', 'cat': 'Canned Goods', 'exp_date': '2025-09-20', 'loc_id': 2},
    {'id': 37, 'name': 'Expired Canned Corn', 'qty': 1, 'unit': 'cans', 'cat': 'Canned Goods', 'exp_date': '2024-05-01', 'loc_id': 6}, # Still categorized
    # Sauces, Oils
    {'id': 10, 'name': 'pasta sauce', 'qty': 1, 'unit': 'jar', 'cat': 'Sauces & Oils', 'exp_date': '4/30/2025', 'loc_id': 2},
    {'id': 35, 'name': 'Olive Oil (Extra Virgin)', 'qty': 350, 'unit': 'ml', 'cat': 'Sauces & Oils', 'exp_date': '2026-06-30', 'loc_id': 2},
    # Dairy & Alternatives
    {'id': 11, 'name': 'Milk (Nonfat)', 'qty': 750, 'unit': 'ml', 'cat': 'Dairy', 'exp_date': '2025-04-14', 'loc_id': 1},
    {'id': 12, 'name': 'Eggs', 'qty': 8, 'unit': 'pcs', 'cat': 'Eggs', 'exp_date': '2025-04-10', 'loc_id': 1}, # Eggs often separate category
    {'id': 13, 'name': 'Cheddar Cheese', 'qty': 250, 'unit': 'g', 'cat': 'Dairy', 'exp_date': '2025-04-25', 'loc_id': 1},
    {'id': 14, 'name': 'Greek Yogurt', 'qty': 20, 'unit': 'carton', 'cat': 'Dairy', 'exp_date': '2025-04-26', 'loc_id': 1}, # Corrected spelling
    {'id': 15, 'name': 'Butter', 'qty': 200, 'unit': 'g', 'cat': 'Dairy', 'exp_date': '2025-05-30', 'loc_id': 1},
    {'id': 16, 'name': 'Almond Milk', 'qty': 900, 'unit': 'ml', 'cat': 'Dairy Alternatives', 'exp_date': '2025-05-05', 'loc_id': 9},
    # Plant-Based Protein
    {'id': 28, 'name': 'Tofu (Firm)', 'qty': 350, 'unit': 'g', 'cat': 'Plant-Based Protein', 'exp_date': '2025-06-18', 'loc_id': 1},
    # Spices & Seasonings
    {'id': 41, 'name': 'Salt (Table)', 'qty': 300, 'unit': 'g', 'cat': 'Spices & Seasonings', 'exp_date': '2028-12-31', 'loc_id': 5},
    {'id': 42, 'name': 'Black Pepper (Ground)', 'qty': 50, 'unit': 'g', 'cat': 'Spices & Seasonings', 'exp_date': '2026-12-01', 'loc_id': 5},
    {'id': 43, 'name': 'Paprika', 'qty': 40, 'unit': 'g', 'cat': 'Spices & Seasonings', 'exp_date': '2026-09-01', 'loc_id': 5},
    # Condiments
    {'id': 44, 'name': 'Ketchup', 'qty': 250, 'unit': 'ml', 'cat': 'Condiments', 'exp_date': '2025-09-15', 'loc_id': 9},
    {'id': 45, 'name': 'Soy Sauce', 'qty': 150, 'unit': 'ml', 'cat': 'Condiments', 'exp_date': '2026-07-20', 'loc_id': 9}
]


Perform data cleaning. 
Create table ingredients and inserting multiple lines of data using engine.

In [ ]:
# Clean the data (standardize dates)
data_to_insert_cleaned = []
for item in data_to_insert_raw:
    new_item = item.copy() # Avoid modifying original dict if reused
    new_item['exp_date'] = standardize_date(item['exp_date'])
    # Optional: Convert quantity to float if you changed column type to REAL/FLOAT
    try:
      new_item['qty'] = float(item['qty'])
    except (ValueError, TypeError):
      new_item['qty'] = None # Or handle error appropriately
    data_to_insert_cleaned.append(new_item)

# --- Execute using a connection ---
try:
    with engine.connect() as connection:
        # Create table
        connection.execute(sql_create_table)

        # Add a column to the table
        #connection.execute(sql_add_column)

        # Insert cleaned data
        # Check if data list is not empty before executing insert
        if data_to_insert_cleaned:
             connection.execute(sql_insert, data_to_insert_cleaned)
        else:
             print("No data to insert after cleaning.")

        # Commit the transaction
        connection.commit()
    print("Ingredients table created (if not exists) and data inserted successfully!")

except sqlalchemy.exc.IntegrityError as e:
     print(f"An Integrity Error occurred (potentially a Foreign Key violation if enabled): {e}")
     # Consider rolling back if necessary, though 'with' block handles disconnect
     # connection.rollback() # Usually not needed with 'with engine.connect()' failure

except Exception as e:
    print(f"An unexpected error occurred: {e}")
    # If the error persists, printing the specific exception type and message is helpful.
    if hasattr(e, 'orig'):
         print(f"Original DBAPI Error: {e.orig}")

## Convert conversation into structured JSON data

In [ ]:
# --- 1. Define the Function/Tool Schema ---
# This describes the structure Gemini should fill if it extracts preferences.
# Based on the JSON structure from the previous prompt.

extract_recipe_preferences = {
    # Gemini's API typically expects a list of tools, each with function_declarations
    "function_declarations": [
        {
            "name": "extract_recipe_preferences", # The name your code will look for
            "description": "Extracts detailed recipe preferences and constraints from a user conversation and formats them into a structured object.",
            "parameters": {
                "type": "OBJECT", # The overall structure is an object
                "properties": {
                    # --- Map each JSON field to a property ---
                    "queryKeywords": {
                        "type": "ARRAY",
                        "description": "List of general keywords or descriptive terms for the desired dish (e.g., ['quick', 'healthy', 'comfort food', 'spicy', 'salad']). Extracted from general descriptions.",
                        "items": { "type": "STRING" }
                    },
                    "cuisine": {
                        "type": "ARRAY",
                        "description": "List of preferred cuisines (e.g., ['Italian', 'Mexican', 'Thai']).",
                        "items": { "type": "STRING" }
                    },
                    "mealType": {
                        "type": "ARRAY",
                        "description": "List of suitable meal types (e.g., ['dinner', 'lunch', 'breakfast', 'snack', 'appetizer', 'dessert']).",
                        "items": { "type": "STRING" }
                    },
                    "totalMaxTimeMinutes": {
                        "type": "INTEGER",
                        "description": "Maximum total time (prep + cook) in minutes allowed for the recipe. Extract the lowest applicable number if multiple are mentioned or implied (e.g., 'under an hour' -> 60, '30-45 mins' -> 45). Use null if not mentioned.",
                        # Add nullable: True if the model API supports it explicitly,
                        # otherwise rely on the LLM omitting the field or using a sentinel value if not found
                        "nullable": True # Check if supported by specific Gemini version/SDK
                    },
                    "cookingMethods": {
                        "type": "ARRAY",
                        "description": "List of preferred or acceptable cooking methods (e.g., ['bake', 'stir-fry', 'grill', 'roast', 'instant pot', 'slow cooker', 'no-cook']).",
                        "items": { "type": "STRING" }
                    },
                    "includedIngredients": {
                        "type": "ARRAY",
                        "description": "List of specific ingredients the Chef *wants* to use or definitely include.",
                        "items": { "type": "STRING" }
                    },
                    "excludedIngredients": {
                        "type": "ARRAY",
                        "description": "List of specific ingredients the Chef explicitly wants to *exclude* (due to dislike, not allergy).",
                        "items": { "type": "STRING" }
                    },
                    "allergies": {
                        "type": "ARRAY",
                        "description": "List of ingredients the Chef is allergic to.",
                        "items": { "type": "STRING" }
                    },
                    "dietaryRestrictions": {
                        "type": "ARRAY",
                        "description": "List of specific dietary restrictions (e.g., ['vegetarian', 'vegan', 'gluten-free', 'keto', 'paleo', 'low-carb', 'low-fat', 'dairy-free']).",
                        "items": { "type": "STRING" }
                    },
                    "texturePreferences": {
                        "type": "ARRAY",
                        "description": "List of desired textures (e.g., ['creamy', 'crunchy', 'smooth', 'chewy']).",
                        "items": { "type": "STRING" }
                    },
                    "flavorProfile": {
                        "type": "ARRAY",
                        "description": "List of desired flavor profiles or characteristics (e.g., ['spicy', 'savory', 'sweet', 'tangy', 'umami', 'rich', 'light']).",
                        "items": { "type": "STRING" }
                    },
                    "notes": {
                        "type": "STRING",
                        "description": "Any other specific requests, nuances, or contextual information mentioned by the Chef that doesn't fit neatly into the other categories.",
                        "nullable": True # Allow notes to be null/omitted
                    }
                },
                # Optional: Define which parameters are absolutely required
                # "required": ["mealType"] # Example: if you always need a meal type
            }
        }
    ]
}


# 3. Implementing the Model

## a) Set environment

In [ ]:
genai.configure(api_key=GOOGLE_API_KEY)

## B) Define Function Call Tools

In [ ]:
# 2. Define Tools (if they are called locally later)

import google.generativeai as genai # <-- **MOST IMPORTANT FIX: ADD THIS IMPORT**
from google.generativeai.types import GenerationConfig, Tool # Import necessary types

def find_all_ingredients(*args, **kwargs):
    print("Called find_all_ingredients")
    connection_string = f'sqlite:///{db_path}' # Note triple slash for absolute path
    print(f"Attempting to create engine for: {connection_string}")

    try:
        # Create the engine (defines how to connect)
        engine = create_engine(connection_string)
    
        # Establish the actual connection (locates DB and connects)
        with engine.connect() as connection: # 'connection' is analogous to 'db' in sqlite3
            print("Database connection successful (via engine).")
    
            # --- Step 2: Execute commands (SQLAlchemy often combines cursor creation) ---
            # You usually don't create an explicit cursor object like in sqlite3.
            # You execute directly using the connection object.
            print("Executing command...")
            result = connection.execute(text("SELECT ingredient_name FROM ingredients"))
            version = result.scalar()
            print(f"SQLite version: {version}")
            return {"ingredients": [row[0] for row in result.fetchall()]}
          
        print("Database connection closed (automatically by 'with' block).")
        
    
    except sqlalchemy.exc.SQLAlchemyError as e:
         print(f"Database error using engine for {connection_string}: {e}")
    except Exception as e:
         print(f"An unexpected error occurred: {e}")
     


def find_ingredients_by_category(*args, **kwargs):
    print(f"Called find_ingredients_by_category with args: {args}, kwargs: {kwargs}")
    connection_string = f'sqlite:///{db_path}' # Note triple slash for absolute path
    print(f"Attempting to create engine for: {connection_string}")

    try:
        # Create the engine (defines how to connect)
        engine = create_engine(connection_string)
    
        # Establish the actual connection (locates DB and connects)
        with engine.connect() as connection: # 'connection' is analogous to 'db' in sqlite3
            print("Database connection successful (via engine).")
    
            # --- Step 2: Execute commands (SQLAlchemy often combines cursor creation) ---
            # You usually don't create an explicit cursor object like in sqlite3.
            # You execute directly using the connection object.
            print("Executing command...")
            category = kwargs.get('category', 'unknown')
            result = connection.execute(text("SELECT ingredient_name FROM ingredients WHERE category = ?", (category,)))        
            version = result.scalar()
            print(f"SQLite version: {version}")
            return {"category": category, "ingredients": [row[0] for row in result.fetchall()]}
   
        print("Database connection closed (automatically by 'with' block).")
    
    except sqlalchemy.exc.SQLAlchemyError as e:
         print(f"Database error using engine for {connection_string}: {e}")
    except Exception as e:
         print(f"An unexpected error occurred: {e}")


def locate_ingredient(*args, **kwargs):
    print(f"Called locate_ingredient with args: {args}, kwargs: {kwargs}")
    ingredient_name = kwargs.get('ingredient_name', 'unknown')
    connection_string = f'sqlite:///{db_path}' # Note triple slash for absolute path
    print(f"Attempting to create engine for: {connection_string}")

    try:
        # Create the engine (defines how to connect)
        engine = create_engine(connection_string)
    
        # Establish the actual connection (locates DB and connects)
        with engine.connect() as connection: # 'connection' is analogous to 'db' in sqlite3
            print("Database connection successful (via engine).")
    
            # --- Step 2: Execute commands (SQLAlchemy often combines cursor creation) ---
            # You usually don't create an explicit cursor object like in sqlite3.
            # You execute directly using the connection object.
            print("Executing command...")
            category = kwargs.get('category', 'unknown')
            result = connection.execute(text("""
                SELECT locations.location_name
                FROM ingredients
                JOIN locations ON ingredients.location_id = locations.location_id
                WHERE ingredients.ingredient_name = ?
            """, (ingredient,)))      
            version = result.scalar()
            print(f"SQLite version: {version}")
            return {"ingredient": ingredient_name, "location": [row[0] for row in result.fetchall()]}
   
        print("Database connection closed (automatically by 'with' block).")
    
    except sqlalchemy.exc.SQLAlchemyError as e:
         print(f"Database error using engine for {connection_string}: {e}")
    except Exception as e:
         print(f"An unexpected error occurred: {e}")

## C) Define the SCHEMAS for the tools

In [ ]:
# Define the SCHEMAS for the tools (this is what Gemini needs)
# Replace these simple examples with your actual detailed schemas
tools_schema = [
    {
        "function_declarations": [
            {
                "name": "find_all_ingredients",
                "description": "Get a list of all available ingredients.",
                "parameters": {} # No parameters needed for this example
            },
            {
                "name": "find_ingredients_by_category",
                "description": "Find ingredients within a specific category (e.g., dairy, produce).",
                "parameters": {
                    "type": "OBJECT",
                    "properties": {
                        "category": {"type": "STRING", "description": "The category to search within (e.g., 'dairy', 'produce')"}
                    },
                    "required": ["category"]
                }
            },
            {
                "name": "locate_ingredient",
                "description": "Find the storage location of a specific ingredient.",
                "parameters": {
                    "type": "OBJECT",
                    "properties": {
                        "ingredient_name": {"type": "STRING", "description": "The name of the ingredient to locate."}
                    },
                    "required": ["ingredient_name"]
                }
            }
        ]
    }
]
# Convert schema list to Tool object (recommended)
# Note: Depending on SDK version, you might pass the list directly in some cases.
# Creating a single Tool object containing multiple function declarations is common.
ingredient_db_tools = Tool(function_declarations=tools_schema[0]['function_declarations'])

## d) Define other variables (system instruction, few-shot prompting, chat_conversation)

In [ ]:
# 4. Define other variables
RECIPE_RECOMD_BOT_SYSINT = (
    "system",  # 'system' indicates the message is a system instruction.
    "You are a helpful recipe recommender system specializing in quick, new, creative meals."
    "If the conversation does not involve making a meal, please steer the chef back to that conversation."
    "\n\n"
    "He/She would tell you what dish they want to make as a general idea (i.e., preferences)." 
    "To convert the chef's preference into structured JSON data, use extract_recipe_preferences and few-shot prompting for examples." 
    "If you are missing crucial information from the chef, ask clarifying questions."
    "Pay extra attention to allegies, restrictions, or exclusions."
    "\n\n"
    "You would use the JSON input as criteria and look up the chef's ingredient inventory database." 
    "and apply chain-of-thought reasoning as a technique for recommendation."
    "When querying the ingredients, prioritize using the earliest expiration date ingredients."
    "Find the most matched recipe and display it to the chef for each dish in a conversation."
    "Please display the dish's picture and the recipe text."
    "\n\n"
    "To find all available ingredients for the recommendation, call find_all_ingredients." 
    "To emphasize or exclude certain ingredient(s) or category, call find_ingredients_by_category."
    "List the location of each ingredient with a parenthesis next to the ingredient by calling locate_ingredient."
    "\n\n"
    "The final response is a recipe recommendation in text and image format. Ask if it looks and sounds good."
    "If so, thank the Chef and hope the dish turns out well. Otherwise, ask what aspects should be changed.",
)

few_shot_prompt = """Parse chef's meal preference into valid JSON for confirmation:

EXAMPLE:
I want a meal with balanced nutritions. Soup sounds good to me today. No specific cuisine preference. 
Make it suitable for the Spring season. Either meat or vegetarian is OK. Keep the spice level low-medium. 
This is for dinner.
JSON Response:
```
{
"key": "balanced nutritions",
"type": "soup",
"meal": "dinner",
"season": "spring",
"spice": "low-medium",
"exclusion": "NULL"
}
```

EXAMPLE:
I want a breakfast that takes at most 20 minutes to prepare and contains three food groups: protein, vegetables/fruit, and complex carbohydrates. 
I like my breakfast cooked in an air fryer. I want avocado in it. Please don't add raw sugar to it.
JSON Response:
```
{
"total time": "20 minutes",
"meal": "breakfast",
"method": "air fryer",
"ingredients": ["complex carbohydrate", "avocado"],
"exclusion": "raw sugar"
}
```

EXAMPLE:
Please recommend a dessert recipe. Some of my favorite dessert ingredients are chocolate and fruit that goes well with chocolate. 
I prefer the chocolate taste in the dessert to be medium-level strong. I prefer soft texture of a dessert but not to the level of 
fluidy like as in a drink. 
JSON Response:
```
{"ingredient": ["chocolate", "fruit goes well with chocolate"],
"chocolate taste": "medium level",
"texture": ["soft", "not liquidy"],
"exclusion": "runny texture"
}
```
"""

chef_conversation = """
Assistant: Hi Chef! What are you thinking of making today?
Chef: Hey! I need a dinner recipe. I was thinking maybe something Mediterranean?
Assistant: Sounds good! Any ingredients you definitely want to use or avoid?
Chef: Let's use chicken breast. And please, absolutely no mushrooms - I really dislike them. Also, my partner is allergic to shellfish, so none of that.
Assistant: Got it - chicken breast, Mediterranean style, no mushrooms, no shellfish. Any dietary needs?
Chef: We're trying to eat relatively low-carb this week.
Assistant: Okay, low-carb Mediterranean chicken dinner. How much time do you have?
Chef: Hmm, preferably under an hour total. Maybe something baked or grilled?
Assistant: Perfect. Anything else? Like specific flavors or textures?
Chef: Not really, just something flavorful and not too complicated.
"""


## e) Code Execution

In [ ]:
# --- Code Execution ---
try:
    # 1. Create GenerationConfig (using the imported class directly)
    generation_config = GenerationConfig(
        temperature=0.8,
        top_p=0.9,
        max_output_tokens=500
    )

    # 2. Instantiate the GenerativeModel (using correct model name)
    model = genai.GenerativeModel(
        model_name='gemini-2.0-flash', 
        generation_config=generation_config,
        tools=ingredient_db_tools, # <-- Pass the formatted Tool object/schema list
        system_instruction=RECIPE_RECOMD_BOT_SYSINT
    )

    # 3. Prepare content (handle few-shot prompt if present)
    content_to_send = []
    if few_shot_prompt:
        # Assuming few_shot_prompt is formatted correctly (e.g., list of Content objects)
        content_to_send.extend(few_shot_prompt) # Use extend for lists
    # Add the current chef conversation
    content_to_send.append(chef_conversation) # Append the string prompt


    # 4. Generate content using the model object's method
    response = model.generate_content(
        contents=content_to_send
    )

    # 5. Process the response robustly
    print("--- Full Response ---")
    print(response)
    print("--- Checking Response ---")

    # Safer check for function call
    candidate = response.candidates[0]
    first_part = candidate.content.parts[0]

    if hasattr(first_part, 'function_call') and first_part.function_call.name:
        function_call = first_part.function_call
        print(f"Function call detected: {function_call.name}")
        # Optional: Call the actual Python function if needed for next step
        # function_to_call = globals()[function_call.name] # Get function object by name
        # function_args = dict(function_call.args)
        # function_response = function_to_call(**function_args)
        # print(f"Function response: {function_response}")
        # You might then send this function_response back to the model
    elif hasattr(first_part, 'text'):
         print(f"Generated text: {first_part.text}") # Access text correctly
    else:
        print("Response did not contain text or a function call in the expected format.")


except AttributeError as e:
    # Catch the specific error if imports are still wrong
    print(f"An AttributeError occurred - check imports and names: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

## Make the model into a chatbot and remember the conversation

In [ ]:
try:
    # Start the chat
    chat_session = model.start_chat(history=[]) # Start with empty history

    # Send the first message
    user_message = "I need to cook a dinner."
    print(f"User: {user_message}")

    response = chat_session.send_message(user_message) # Capture the response

    # Process and print the response
    print(f"Model: {response.text}")

except Exception as e:
    print(f"An error occurred: {e}")